In [70]:
import pandas as pd
import numpy as np
import os

# Allow pretty-display of multiple variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Adjust the way pandas DataFrames are displayed to fit more columns
#pd.reset_option('display.max_colwidth')
#pd.options.display.max_columns = 12
#pd.options.display.max_colwidth = 5
#pd.set_option('display.max_row', 1000)


#
# User-defined CONSTANTS
#
YEAR = 2018
LEAP_YEAR = False # TODO: determine automatically
VERBOSE = False # additional output for debugging and error-checking

#
# Global CONSTANTS
#
if not LEAP_YEAR: 
    HOURS_IN_YEAR = 8760
    DAYS_IN_FEB = 28
else: 
    HOURS_IN_YEAR = 8760 + 24
    DAYS_IN_FEB = 29
    
DAYS_IN_JAN = 31

# DateTimeIndex for resampling time series
START_DATE = str(YEAR) + '-01-01' 
DTI15 = pd.date_range(START_DATE, periods=4*HOURS_IN_YEAR, freq='15min')
DTI30 = pd.date_range(START_DATE, periods=2*HOURS_IN_YEAR, freq='30min')
DTI60 = pd.date_range(START_DATE, periods=HOURS_IN_YEAR, freq='60min')

# DST starts March 26, 2017 / March 25, 2018 / March 31, 2019
DST_START_DAY_MAR = {
    2017 : 26,
    2018 : 25,
    2019 : 31
}

DST_END_DAY_OCT = {
    2017 : 29,
    2018 : 28,
    2019 : 27
}

DST_BOUNDARY_START = pd.Timestamp(year=YEAR, month=3, day=DST_START_DAY_MAR[YEAR], hour=2)
DST_BOUNDARY_END = pd.Timestamp(year=YEAR, month=10, day=DST_END_DAY_OCT[YEAR], hour=2)


# Time Intervals: Total Load
LOAD_FREQ = {
    'AT' : 15, 'BE' : 15, 'DE' : 15, 'HU' : 15, 'LU' : 15, 'NL' : 15,
    'IE' : 30, 'UK' : 30, #   'CY' : 30,
    'BG' : 60, 'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60, 'FI' : 60,
    'FR' : 60, 'GR' : 60, 'HR' : 60, 'IT' : 60, 'LT' : 60, 'LV' : 60,
    'PL' : 60, 'PT' : 60, 'RO' : 60, 'SE' : 60, 'SI' : 60, 'SK' : 60
}

# Time Intervals: Generation Forecasts
GEN_FORECAST_FREQ = {
    'AT' : 15, 'DE' : 15, 'HU' : 15, 'NL' : 15, #   'CY' : 30,
    'BE' : 60, 'BG' : 60, 'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60,
    'FI' : 60, 'FR' : 60, 'HR' : 60, 'IE' : 60, 'IT' : 60, 'LT' : 60,
    'LU' : 60, 'LV' : 60, 'GR' : 60, 'PL' : 60, 'PT' : 60, 'RO' : 60,
    'SE' : 60, 'SI' : 60, 'SK' : 60, 'UK' : 60
}

# Time Intervals: Actual Generation per Production Type
GEN_ACTUAL_FREQ = {
    'AT' : 15, 'DE' : 15, 'HU' : 15, 'NL' : 15, #   'CY' : 30,
    'IE' : 30, 'UK' : 30,
    'BE' : 60, 'BG' : 60, 'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60,
    'FI' : 60, 'FR' : 60, 'GR' : 60, 'HR' : 60, 'IT' : 60, 'LT' : 60,
    'LU' : 60, 'LV' : 60, 'PL' : 60, 'PT' : 60, 'RO' : 60, 'SE' : 60,
    'SI' : 60, 'SK' : 60
}

# Time Intervals: Total Imbalance
IMBALANCE_FREQ = {
    'AT' : 15, 'BE' : 15, 'DE' : 15, 'HU' : 15, 'IT' : 15, 'LU' : 15,
    'NL' : 15, 'SK' : 15,
    'FR' : 30, 'UK' : 30,
    'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60, 'FI' : 60, 'LV' : 60, 
    'LT' : 60, 'PL' : 60, 'PT' : 60, 'RO' : 60, 'SI' : 60, 'SE' : 60    
}

# Set of countries with COMPLETE solar data
# Warning: values can change from year to year
HAS_SOLAR_DATA = {'AT', 'BE', 'BG', 'CZ', 'DE', 'DK', 'ES', 'FR', 
                  'GR', 'LT', 'NL', 'PT', 'RO', 'SI', 'SK', 'UK'}

# Set of countries with COMPLETE offshore wind data
# Warning: values can change from year to year
# For example, Cyprus may develop offshore
HAS_OFFSHORE_WIND_DATA = {'BE', 'DE', 'DK', 'NL', 'UK'}

HAS_NO_BALANCE_DATA = {'BG', 'HR', 'GR', 'IE'} 
HAS_EXTRA_BALANCE_COL = {'DK', 'FI', 'IT', 'SE'}

In [71]:
#
# TODO: define class
#
        
def load_data_from_csv(country = "DE") :

    # directory where raw csv files reside grouped by year then country
    path = "data" + os.sep + str(YEAR) + os.sep + country + os.sep 
    tail = str(YEAR) + "01010000-" + str(YEAR+1) + "01010000.csv"
    
    vre_forecast_csv_filename = path + "Generation Forecasts for Wind and Solar_" + tail
    print("Loading", vre_forecast_csv_filename)
    vre_forecast = pd.read_csv(vre_forecast_csv_filename, usecols=[0,1,4,7])
    vre_forecast.columns = ['Time','Forecast.Solar','Forecast.Offshore','Forecast.Onshore']
    drop_dst_rows(vre_forecast, GEN_FORECAST_FREQ[country])
    if (country not in HAS_SOLAR_DATA) : vre_forecast['Forecast.Solar'] = 0
    if (country not in HAS_OFFSHORE_WIND_DATA) : vre_forecast['Forecast.Offshore'] = 0
    #vre_forecast.dropna(axis=1)
    vre_forecast = convert_to_hourly_mean(vre_forecast, GEN_FORECAST_FREQ[country]) 
    if (VERBOSE) : print(vre_forecast)
    
    actual_generation_csv_filename = path + "Actual Generation per Production Type_" + tail
    print("Loading", actual_generation_csv_filename)
    actual_generation = pd.read_csv(actual_generation_csv_filename, usecols=[19,21,22])
    actual_generation.columns = ['Actual.Solar', 'Actual.Offshore', 'Actual.Onshore']
    drop_dst_rows(actual_generation, GEN_ACTUAL_FREQ[country])
    if (country not in HAS_SOLAR_DATA) : actual_generation['Actual.Solar'] = 0
    if (country not in HAS_OFFSHORE_WIND_DATA) : actual_generation['Actual.Offshore'] = 0
    #actual_generation.dropna(axis=1)
    actual_generation = convert_to_hourly_mean(actual_generation, GEN_ACTUAL_FREQ[country])
    if (VERBOSE) : print(actual_generation)
    
    total_load_csv_filename = path + "Total Load - Day Ahead _ Actual_" + tail
    print("Loading", total_load_csv_filename)
    total_load = pd.read_csv(total_load_csv_filename, usecols=[1,2])
    total_load.columns = ['Forecast.Load', 'Actual.Load']
    drop_dst_rows(total_load, LOAD_FREQ[country])
    total_load = convert_to_hourly_mean(total_load, LOAD_FREQ[country])
    if (VERBOSE) : print(total_load)
        
    if (country in HAS_NO_BALANCE_DATA) : 
        total_load['Imbalance'] = 0 # not the cleanest way to go about it, but gets the job done 
        return pd.concat([vre_forecast, actual_generation, total_load], axis=1)
        
    imbalance_csv_filename = path + "Imbalance_" + tail
    print("Loading", imbalance_csv_filename)
    if (country in HAS_EXTRA_BALANCE_COL) : 
        imbalance = pd.read_csv(imbalance_csv_filename, usecols=[4], na_values=['n/e'])
    else : 
        imbalance = pd.read_csv(imbalance_csv_filename, usecols=[3], na_values=['n/e'])
    imbalance.columns = ['Imbalance'] # Total Imbalance Volume in MWh
    imbalance.fillna(0, inplace=True)    
    drop_dst_rows(imbalance, IMBALANCE_FREQ[country])
    imbalance = convert_to_hourly_sum(imbalance, IMBALANCE_FREQ[country])
    if (VERBOSE) : print(imbalance)

    return pd.concat([vre_forecast, actual_generation, total_load, imbalance], axis=1)


#
# Handle DST by deleting extra (empty) row(s)
# The DST start date and time is hard-coded to save time
# and because we are only concerned with 2017, 2018, and 2019
#
def drop_dst_rows(df, freq) :

    if (VERBOSE) : print("DST start date is March ", DST_START_DAY_MAR[YEAR])
    start_row = 24 * (DAYS_IN_JAN + DAYS_IN_FEB + (DST_START_DAY_MAR[YEAR] - 1))
    start_row = start_row + 2   # 2am                   
    if (freq == 15) : # 15 minute intervals
        start_row = start_row * 4
        end_row = start_row + 4
    elif (freq == 30) : # 30 minute intervals
        start_row = start_row * 2
        end_row = start_row + 2
    elif (freq == 60) : # hourly intervals
        end_row = start_row + 1
    # else : # raise exception - invalid frequency
    
    if (VERBOSE) : print("DST row(s):", df[start_row:end_row])
    df.drop(df.index[start_row:end_row], inplace=True)    
        
    return


def convert_to_hourly_mean(df, freq) :

    if (freq == 15) : df.index = DTI15   # 15 min intervals        
    elif (freq == 30) : df.index = DTI30 # 30 min intervals
    elif (freq == 60) : df.index = DTI60 # 60 min intervals
    # else : # raise exception - invalid frequency

    return df.resample('H').mean()   


def convert_to_hourly_sum(df, freq) :

    if (freq == 15) : df.index = DTI15   # 15 min intervals        
    elif (freq == 30) : df.index = DTI30 # 30 min intervals
    elif (freq == 60) : df.index = DTI60 # 60 min intervals
    # else : # raise exception - invalid frequency

    return df.resample('H').sum()   


def map_to_raw_timestamp(t) :
        
    if (t < DST_BOUNDARY_START) : 
        return (str(t) + " - " + str(t + pd.Timedelta('1 hour')))
    elif (t < DST_BOUNDARY_END) :
        return (str(t + pd.Timedelta('1 hour')) + " - " + str(t + pd.Timedelta('2 hours')))
    elif (t == DST_BOUNDARY_END) : # Extra added hour from DST transition
        return ("*" + str(t) + " - " + str(t + pd.Timedelta('1 hour')) + "*")
    else :
        return (str(t) + " - " + str(t + pd.Timedelta('1 hour')))
              
#TODO: Output string format: "01.01.2018 23:00 - 02.01.2018 00:00"
#index = 7200    # Test cases: 1990, 7200, 8754 
#for i in range(index, index+6): print(map_to_raw_timestamp(DTI60[i]))

In [72]:
def calc_critical_hours(country = "DE") :

    hours = load_data_from_csv(country) 

    # Set to zero if either column is missing 
    # Solar and Offshore are handled above via HAS_SOLAR and HAS_OFFSHORE
    if (not 'Forecast.Onshore' in hours.columns or hours['Forecast.Onshore'].isnull().all()) :
        hours['Forecast.Onshore'] = 0
        hours['Actual.Onshore'] = 0

    # Combine Onshore and Offshore
    hours.insert(4, 'Forecast.Wind', hours['Forecast.Offshore'] + hours['Forecast.Onshore'])
    hours.drop(['Forecast.Offshore','Forecast.Onshore'], axis=1, inplace=True) #hours.head(4)
    hours.insert(3, 'Actual.Wind', hours['Actual.Offshore'] + hours['Actual.Onshore'])
    hours.drop(['Actual.Offshore','Actual.Onshore'], axis=1, inplace=True) #hours.head(4)

    # Calculate VRE
    hours.insert(4, 'Forecast.VRE', hours['Forecast.Solar'] + hours['Forecast.Wind'])
    hours.insert(5, 'Actual.VRE', hours['Actual.Solar'] + hours['Actual.Wind'])

    # Calculate Residual Load and Delta Residual Load
    hours['Forecast.ResLoad'] = hours['Forecast.Load'] - hours['Forecast.VRE']
    hours['Actual.ResLoad'] = hours['Actual.Load'] - hours['Actual.VRE']
    hours['Delta.ResLoad'] = hours['Actual.ResLoad'] - hours['Forecast.ResLoad']
    
    # Add new column with numeric values
    hours['Index'] = np.arange(1, len(hours) + 1) # print(hours.tail(4))
    
    # Critical hours
    critical_hours = hours.sort_values(by=['Delta.ResLoad'],ascending=False).head(10)
    critical_hours['Hour'] = list(critical_hours.index) # new column of sorted timestamps
    critical_hours['Hour'] = critical_hours['Hour'].apply(map_to_raw_timestamp)

    # Reset dataframe index to [1..8760]
    critical_hours.index = list(critical_hours['Index']) 
    critical_hours.drop(columns='Index',inplace=True)
    
    # Export to csv 
    filename = "Critical_Hours_" + country + "_" + str(YEAR) + ".csv" #print(filename)
    critical_hours.to_csv(filename) 
    return critical_hours

In [73]:
for country in LOAD_FREQ : # iterate thru keys in dictionary (in random order)
    ch = calc_critical_hours(country)
    ch

Loading data/2018/FR/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/FR/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/FR/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/FR/Imbalance_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Imbalance,Forecast.ResLoad,Actual.ResLoad,Delta.ResLoad,Hour
55,0.0,0.0,11496.0,8298.0,11496.0,8298.0,54150.0,56555.0,-2410.0,42654.0,48257.0,5603.0,2018-01-03 06:00:00 - 2018-01-03 07:00:00
8634,30.0,0.0,1258.0,1111.0,1288.0,1111.0,64050.0,68566.0,124.0,62762.0,67455.0,4693.0,2018-12-26 17:00:00 - 2018-12-26 18:00:00
2202,1279.0,704.0,4630.0,4744.0,5909.0,5448.0,47800.0,52015.0,2058.0,41891.0,46567.0,4676.0,2018-04-02 18:00:00 - 2018-04-02 19:00:00
18,24.0,48.0,9858.0,9373.0,9882.0,9421.0,52600.0,56686.0,-1859.0,42718.0,47265.0,4547.0,2018-01-01 17:00:00 - 2018-01-01 18:00:00
8375,0.0,0.0,8235.0,4879.0,8235.0,4879.0,68900.0,70056.0,303.0,60665.0,65177.0,4512.0,2018-12-15 22:00:00 - 2018-12-15 23:00:00
16,951.0,642.0,9490.0,7730.0,10441.0,8372.0,50950.0,53357.0,-2389.0,40509.0,44985.0,4476.0,2018-01-01 15:00:00 - 2018-01-01 16:00:00
391,0.0,0.0,9558.0,9576.0,9558.0,9576.0,61900.0,66345.0,-1495.0,52342.0,56769.0,4427.0,2018-01-17 06:00:00 - 2018-01-17 07:00:00
1627,111.0,95.0,4775.0,4383.0,4886.0,4478.0,60700.0,64632.0,1658.0,55814.0,60154.0,4340.0,2018-03-09 18:00:00 - 2018-03-09 19:00:00
7674,153.0,153.0,1645.0,1624.0,1798.0,1777.0,57550.0,61829.0,-945.0,55752.0,60052.0,4300.0,2018-11-16 17:00:00 - 2018-11-16 18:00:00
8633,646.0,424.0,892.0,844.0,1538.0,1268.0,60500.0,64442.0,129.0,58962.0,63174.0,4212.0,2018-12-26 16:00:00 - 2018-12-26 17:00:00


Loading data/2018/DK/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/DK/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/DK/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/DK/Imbalance_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Imbalance,Forecast.ResLoad,Actual.ResLoad,Delta.ResLoad,Hour
694,0.0,0.0,4777.0,3958.0,4777.0,3958.0,4146.0,4331.0,130.0,-631.0,373.0,1004.0,2018-01-29 21:00:00 - 2018-01-29 22:00:00
1790,350.0,274.0,4859.0,3986.0,5209.0,4260.0,4953.0,4974.0,131.0,-256.0,714.0,970.0,2018-03-16 13:00:00 - 2018-03-16 14:00:00
1814,364.0,397.0,4730.0,3745.0,5094.0,4142.0,4352.0,4369.0,28.0,-742.0,227.0,969.0,2018-03-17 13:00:00 - 2018-03-17 14:00:00
1815,318.0,354.0,4419.0,3539.0,4737.0,3893.0,4251.0,4318.0,151.0,-486.0,425.0,911.0,2018-03-17 14:00:00 - 2018-03-17 15:00:00
6614,384.0,468.0,3657.0,2715.0,4041.0,3183.0,4563.0,4570.0,145.0,522.0,1387.0,865.0,2018-10-03 14:00:00 - 2018-10-03 15:00:00
664,41.0,52.0,4656.0,3789.0,4697.0,3841.0,3739.0,3739.0,46.0,-958.0,-102.0,856.0,2018-01-28 15:00:00 - 2018-01-28 16:00:00
650,0.0,0.0,3853.0,3151.0,3853.0,3151.0,2995.0,3130.0,66.0,-858.0,-21.0,837.0,2018-01-28 01:00:00 - 2018-01-28 02:00:00
7103,0.0,0.0,4415.0,3470.0,4415.0,3470.0,3374.0,3260.0,80.0,-1041.0,-210.0,831.0,2018-10-23 23:00:00 - 2018-10-24 00:00:00
5012,29.0,9.0,1404.0,647.0,1433.0,656.0,3240.0,3284.0,17.0,1807.0,2628.0,821.0,2018-07-28 20:00:00 - 2018-07-28 21:00:00
6592,166.0,189.0,4624.0,3836.0,4790.0,4025.0,4411.0,4462.0,56.0,-379.0,437.0,816.0,2018-10-02 16:00:00 - 2018-10-02 17:00:00


Loading data/2018/BG/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/BG/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/BG/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv


...

,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Imbalance,Forecast.ResLoad,Actual.ResLoad,Delta.ResLoad,Hour
6327,9415.50,7645.75,39557.25,33330.50,48972.75,40976.25,66504.75,71448.75,1353.0,17532.00,30472.50,12940.50,2018-09-21 15:00:00 - 2018-09-21 16:00:00
6326,11892.50,10476.75,40030.75,34542.25,51923.25,45019.00,67061.00,73016.75,1355.0,15137.75,27997.75,12860.00,2018-09-21 14:00:00 - 2018-09-21 15:00:00
6353,4829.25,3469.50,28669.75,20939.00,33499.00,24408.50,53818.50,57065.50,340.0,20319.50,32657.00,12337.50,2018-09-22 17:00:00 - 2018-09-22 18:00:00
6325,13899.50,12434.75,39803.25,35991.75,53702.75,48426.50,68100.50,74838.00,1103.0,14397.75,26411.50,12013.75,2018-09-21 13:00:00 - 2018-09-21 14:00:00
6328,6640.00,5124.50,38588.00,32582.50,45228.00,37707.00,65207.75,69416.75,223.0,19979.75,31709.75,11730.00,2018-09-21 16:00:00 - 2018-09-21 17:00:00
6352,9159.25,7246.25,30463.75,23572.00,39623.00,30818.25,53668.00,56485.75,961.0,14045.00,25667.50,11622.50,2018-09-22 16:00:00 - 2018-09-22 17:00:00
6259,100.25,80.75,16927.50,15043.75,17027.75,15124.50,63119.75,72342.25,790.0,46092.00,57217.75,11125.75,2018-09-18 19:00:00 - 2018-09-18 20:00:00
6324,14852.25,13574.25,38721.25,35711.25,53573.50,49285.50,69158.50,75900.50,392.0,15585.00,26615.00,11030.00,2018-09-21 12:00:00 - 2018-09-21 13:00:00
6329,3613.75,2634.00,37214.25,31581.00,40828.00,34215.00,64253.25,68651.00,1049.0,23425.25,34436.00,11010.75,2018-09-21 17:00:00 - 2018-09-21 18:00:00
8080,551.25,420.25,27375.75,21123.00,27927.00,21543.25,68189.25,72809.75,1045.0,40262.25,51266.50,11004.25,2018-12-03 15:00:00 - 2018-12-03 16:00:00


Loading data/2018/AT/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/AT/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/AT/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/AT/Imbalance_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Imbalance,Forecast.ResLoad,Actual.ResLoad,Delta.ResLoad,Hour
7506,3.0,18.0,1139.0,816.0,1142.0,834.0,6080.0,8987.00,-134.0,4938.0,8153.00,3215.00,2018-11-09 17:00:00 - 2018-11-09 18:00:00
7212,293.0,294.0,2505.0,216.0,2798.0,510.0,6270.0,7029.50,1056.0,3472.0,6519.50,3047.50,2018-10-28 11:00:00 - 2018-10-28 12:00:00
7214,293.0,234.0,2569.0,340.0,2862.0,574.0,5793.0,6538.00,581.0,2931.0,5964.00,3033.00,2018-10-28 13:00:00 - 2018-10-28 14:00:00
7507,0.0,8.0,1189.0,845.0,1189.0,853.0,6080.0,8772.25,88.0,4891.0,7919.25,3028.25,2018-11-09 18:00:00 - 2018-11-09 19:00:00
7213,322.0,302.0,2582.0,313.0,2904.0,615.0,6040.0,6779.00,166.0,3136.0,6164.00,3028.00,2018-10-28 12:00:00 - 2018-10-28 13:00:00
7211,212.0,212.0,2379.0,139.0,2591.0,351.0,6167.0,6933.75,998.0,3576.0,6582.75,3006.75,2018-10-28 10:00:00 - 2018-10-28 11:00:00
7216,152.0,100.0,2427.0,287.0,2579.0,387.0,5660.0,6328.75,821.0,3081.0,5941.75,2860.75,2018-10-28 15:00:00 - 2018-10-28 16:00:00
7217,70.0,47.0,2316.0,373.0,2386.0,420.0,5788.0,6670.75,207.0,3402.0,6250.75,2848.75,2018-10-28 16:00:00 - 2018-10-28 17:00:00
7505,33.0,56.0,1090.0,771.0,1123.0,827.0,6080.0,8626.50,226.0,4957.0,7799.50,2842.50,2018-11-09 16:00:00 - 2018-11-09 17:00:00
7500,255.0,318.0,859.0,863.0,1114.0,1181.0,6080.0,8936.25,45.0,4966.0,7755.25,2789.25,2018-11-09 11:00:00 - 2018-11-09 12:00:00


Loading data/2018/UK/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/UK/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/UK/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/UK/Imbalance_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Imbalance,Forecast.ResLoad,Actual.ResLoad,Delta.ResLoad,Hour
6114,2160.0,1898.5,6221.0,4728.5,8381.0,6627.0,38819.0,55444.5,-151.0,30438.0,48817.5,18379.5,2018-09-12 18:00:00 - 2018-09-12 19:00:00
6442,2813.0,3095.0,10928.0,2630.0,13741.0,5725.0,36962.0,39816.0,-21.0,23221.0,34091.0,10870.0,2018-09-26 10:00:00 - 2018-09-26 11:00:00
7944,0.0,0.0,12480.0,5476.5,12480.0,5476.5,34256.0,37345.0,997.0,21776.0,31868.5,10092.5,2018-11-27 23:00:00 - 2018-11-28 00:00:00
7465,0.0,0.0,9388.0,3625.5,9388.0,3625.5,27751.0,32077.0,83.0,18363.0,28451.5,10088.5,2018-11-08 00:00:00 - 2018-11-08 01:00:00
7436,0.0,0.0,13156.0,6551.5,13156.0,6551.5,43439.0,46177.0,397.0,30283.0,39625.5,9342.5,2018-11-06 19:00:00 - 2018-11-06 20:00:00
8205,0.0,0.0,11752.0,6112.0,11752.0,6112.0,38601.5,41964.0,221.0,26849.5,35852.0,9002.5,2018-12-08 20:00:00 - 2018-12-08 21:00:00
7371,0.0,0.0,13057.0,6121.5,13057.0,6121.5,24495.5,26444.5,100.0,11438.5,20323.0,8884.5,2018-11-04 02:00:00 - 2018-11-04 03:00:00
8379,0.0,0.0,13467.0,6827.5,13467.0,6827.5,29153.5,31338.5,374.0,15686.5,24511.0,8824.5,2018-12-16 02:00:00 - 2018-12-16 03:00:00
7464,0.0,0.0,9972.0,5746.5,9972.0,5746.5,31359.5,35837.0,513.0,21387.5,30090.5,8703.0,2018-11-07 23:00:00 - 2018-11-08 00:00:00
7463,0.0,0.0,10441.0,6110.0,10441.0,6110.0,35417.5,39617.0,888.0,24976.5,33507.0,8530.5,2018-11-07 22:00:00 - 2018-11-07 23:00:00
